In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [2]:
data = pd.read_csv('student_mat.csv', header = 0, sep = ';', na_values='?')

In [3]:
data.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


## Проверим, нет ли пропусков

In [4]:
data.isnull().any()

school        False
sex           False
age           False
address       False
famsize       False
Pstatus       False
Medu          False
Fedu          False
Mjob          False
Fjob          False
reason        False
guardian      False
traveltime    False
studytime     False
failures      False
schoolsup     False
famsup        False
paid          False
activities    False
nursery       False
higher        False
internet      False
romantic      False
famrel        False
freetime      False
goout         False
Dalc          False
Walc          False
health        False
absences      False
G1            False
G2            False
G3            False
dtype: bool

In [5]:
target=data.G3
data.drop('G3', axis=1, inplace=True)

## Выберем категориальные признаки и применим к ним бинарное кодирование

In [6]:
cat_cols=[c for c in data.columns if data[c].dtype.name=='object' and data[c].value_counts().size>2]

In [7]:
cat_cols

['Mjob', 'Fjob', 'reason', 'guardian']

1 способ

In [8]:
dummies=pd.get_dummies(data[cat_cols])

In [9]:
dummies.head()

,Mjob_at_home,Mjob_health,Mjob_other,Mjob_services,Mjob_teacher,Fjob_at_home,Fjob_health,Fjob_other,Fjob_services,Fjob_teacher,reason_course,reason_home,reason_other,reason_reputation,guardian_father,guardian_mother,guardian_other
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


NB Не применяем этот способ после train_test_split, т.к. количество столбцов  в обучающей и тестовой части может быть различным

2 способ

In [10]:
from sklearn.feature_extraction import DictVectorizer

encoder=DictVectorizer(sparse=False)
encoder.fit_transform(data[cat_cols].T.to_dict().values())

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  1.,  0.],
       ..., 
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.]])

NB 1. Этот метод можно применять только к строковому типу данных

   2. DictVectorizer после train_test_split:  __train\_X.reset\_index(drop=True).T.to\_dict().values()__, а не __train\_X.T.to\_dict().values()__ (для test_X аналогично). Т.к. при помещении в словарь  индекс строки исходной таблицы  заменяется обычным порядковым номером. Теряется взаимосвязь с вектором ответов y.

## Выберем порядковые переменные и закодируем их

Порядковые признаки - частный случай категориальных. На них часто нельзя ввести метрику,т.к. расстояние от значения "начальное образование" до значения "среднее образование" по смыслу может быть не равно расстоянию от "среднее образование" до "высшее образование".

В данном примере с порядковыми признаками лучше работать так же, как с количественными.

Далее мы кодируем порядковые признаки только для демонстрации  метода OneHotEncoder, а не для дальнейшей работы.

Выберем бинарные, количественные и порядковые признаки

In [11]:
bin_cols=[c for c in data.columns if  data[c].value_counts().size==2]

In [12]:
bin_cols

['school',
 'sex',
 'address',
 'famsize',
 'Pstatus',
 'schoolsup',
 'famsup',
 'paid',
 'activities',
 'nursery',
 'higher',
 'internet',
 'romantic']

Любым удобным способом заменяем значения бинарных признаков на 0/1.Например, для 'school':

In [13]:
data.school.replace(['GP','MS'],[0,1],inplace=True)

In [14]:
real_cols=['age', 'absences', 'G1' ,'G2']

In [15]:
ordered_cols=list(set(data.columns.values.tolist())-set(cat_cols)-set(real_cols)-set(bin_cols))

In [17]:
ordered_cols

['Dalc',
 'famrel',
 'Walc',
 'studytime',
 'Fedu',
 'goout',
 'health',
 'freetime',
 'Medu',
 'failures',
 'traveltime']

Кодируем порядковые признаки

In [18]:
encoder=preprocessing.OneHotEncoder(handle_unknown = 'ignore')
encoder.fit_transform(data[ordered_cols])

<395x52 sparse matrix of type '<type 'numpy.float64'>'
	with 4345 stored elements in Compressed Sparse Row format>

Посмотрим, как работает метод:

In [19]:
print encoder.n_values_

print encoder.feature_indices_

print encoder.transform([[1, 2, 1, 1, 1, 1,1, 1, 1, 1, 1 ]]).toarray()

[6 6 6 5 5 6 6 6 5 4 5]
[ 0  6 12 18 23 28 34 40 46 51 55 60]
[[ 1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  1.  0.  0.  0.  0.  1.  0.  0.
   0.  0.  1.  0.  0.  0.  1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  1.  0.
   0.  0.  0.  0.  1.  0.  0.  0.  0.  1.  0.  0.  1.  0.  0.  0.]]


NB Метод применяется к типу данных int.

Параметр handle_unknown = 'ignore' применяется, чтобы  новые значения в тестовой выборке(которых не было в обучающей) обозначались 0. Можно при встрече новых значений выдавать ошибку handle_unknown = 'error'